In [ ]:
%matplotlib inline
import numpy as np 
import pandas as pd 

In [12]:
train_df = pd.read_csv('train 2.csv', encoding='latin-1', usecols= ['id', 'CRIMETYPE', 'BEGDATE'])
test_df = pd.read_csv('test 3.csv', encoding='latin-1', usecols= ['id', 'BEGDATE'])
train_df['Date'] = pd.to_datetime(train_df.BEGDATE)
test_df['Date'] = pd.to_datetime(test_df.BEGDATE)

train_df = train_df.drop('BEGDATE', axis=1)
test_df = test_df.drop('BEGDATE', axis=1)


train_df['IsDay'] = 0
train_df.loc[ (train_df.Date.dt.hour > 6) & (train_df.Date.dt.hour < 20), 'IsDay' ] = 1
test_df['IsDay'] = 0
test_df.loc[ (test_df.Date.dt.hour > 6) & (test_df.Date.dt.hour < 20), 'IsDay' ] = 1


train_df['Hour'] = train_df.Date.dt.hour
train_df['Month'] = train_df.Date.dt.month
train_df['Year'] = train_df.Date.dt.year
train_df['Year'] = train_df['Year'] - 2000 # The Algorithm doesn't know the difference. It's just easier to work like that

test_df['Hour'] = test_df.Date.dt.hour
test_df['Month'] = test_df.Date.dt.month
test_df['Year'] = test_df.Date.dt.year
test_df['Year'] = test_df['Year'] - 2000 # The Algorithm doesn't know the difference. It's just easier to work like that


train_df['HourCos'] = np.cos((train_df['Hour']*2*np.pi)/24 )
train_df['MonthCos'] = np.cos((train_df['Month']*2*np.pi)/12 )

test_df['HourCos'] = np.cos((test_df['Hour']*2*np.pi)/24 )
test_df['MonthCos'] = np.cos((test_df['Month']*2*np.pi)/12 )





In [13]:
train_df.sample(1)

,id,CRIMETYPE,Date
31525,85708,BTFV,2010-11-10


In [14]:
train_df.Date.dtype

dtype('<M8[ns]')

In [15]:
train_df.sample(3)

,id,CRIMETYPE,Date,IsDay
30609,11762,BURG,2009-03-30,0
16444,192849,BURG,2013-05-14,0
67294,226954,BTFV,2014-03-19,0


In [16]:
train_df.sample(1)

,id,CRIMETYPE,Date,IsDay,Hour,Month,Year
54914,63826,BTFV,2010-05-11,0,0,5,10


In [17]:
train_df.sample(1)

,id,CRIMETYPE,Date,IsDay,Hour,Month,Year,HourCos,MonthCos
47245,123403,BURG,2011-09-29,0,0,9,11,1.0,-1.836970e-16


In [ ]:

feature_cols = ['IsDay', 'Hour', 'Month', 'Year', 'HourCos',
       'MonthCos']
target_col = 'CRIMETYPE'

In [23]:
test_df.columns

Index(['id', 'Date', 'IsDay', 'Hour', 'Month', 'Year', 'HourCos', 'MonthCos',
       'CRIMETYPE'],
      dtype='object')

In [19]:
train_df['CRIMETYPE'] = np.where(train_df['CRIMETYPE']=='BURG', 1, 0)
test_df['CRIMETYPE'] = np.where(train_df['CRIMETYPE']=='BURG', 1, 0)

In [20]:
train_df.columns

Index(['id', 'CRIMETYPE', 'Date', 'IsDay', 'Hour', 'Month', 'Year', 'HourCos',
       'MonthCos'],
      dtype='object')

In [30]:
feature_cols = ['IsDay', 'Hour', 'Month', 'Year', 'HourCos',
       'MonthCos']
target_col = 'CRIMETYPE'

train_x = train_df[feature_cols]
train_y = train_df[target_col]

test_ids = test_df['id']
test_x = test_df[feature_cols]

In [31]:
type(train_x), type(train_y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [27]:
!pip install xgboost

     |████████████████████████████████| 1.7 MB 1.7 MB/s eta 0:00:01


In [32]:
import xgboost as xgb
train_xgb = xgb.DMatrix(train_x, label=train_y)
test_xgb  = xgb.DMatrix(test_x)

In [33]:

params = {
    'max_depth': 4,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 39,
}

In [34]:
CROSS_VAL = False
if CROSS_VAL:
    print('Doing Cross-validation ...')
    cv = xgb.cv(params, train_xgb, nfold=3, early_stopping_rounds=10, metrics='mlogloss', verbose_eval=True)
    cv

In [37]:
SUBMIT = not CROSS_VAL
if SUBMIT:
    print('Fitting Model ...')
    m = xgb.train(params, train_xgb, 10)
    res = m.predict(test_xgb)
    cols = ['id']
    submission = pd.DataFrame(res)
    submission.insert(0, 'id', test_ids)
    submission.to_csv('submissionvy.csv', index=False)
    print('Done Outputing !')
    print(submission.sample(3))
else:
    print('NOT SUBMITING')


Fitting Model ...
[03:24:41] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Done Outputing !
          id         0         1         2         3         4         5  \
21414  72937  0.578638  0.333257  0.002381  0.002381  0.002381  0.002381   
6308   55257  0.592908  0.322611  0.002283  0.002283  0.002283  0.002283   
22111  99142  0.593933  0.318374  0.002370  0.002370  0.002370  0.002370   

              6         7         8  ...        29        30        31  \
21414  0.002381  0.002381  0.002381  ...  0.002381  0.002381  0.002381   
6308   0.002283  0.002283  0.002283  ...  0.002283  0.002283  0.00